In [1]:
import pypsa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

n = pypsa.Network()
n.import_from_netcdf(path="../FYP-TZ-APG-Hydrogen-Integration/TZ-APG v1/tz-apg-v1_scenario-bau.nc")

INFO:pypsa.network.io:New version 1.0.0 available! (Current: 0.35.2)
INFO:pypsa.network.io:Imported network '' has buses, carriers, generators, global_constraints, links, loads, storage_units


In [11]:
# --- Helper Function ---
def annuity(r, n_years):
    return r / (1 - (1 + r)**(-n_years))

# Common techno-economic assumptions (2030 base year)
WACC = 0.07

# Electrolyser
ely = dict(eff=0.72, capex=550_000, fixed=10_000, life=20, var=0.8)
ely["capital_cost"] = ely["capex"] * annuity(WACC, ely["life"]) + ely["fixed"]

# H2 → Power turbine
h2p = dict(eff=0.60, capex=800_000, fixed=26_000, life=40, var=2.8)
h2p["capital_cost"] = h2p["capex"] * annuity(WACC, h2p["life"]) + h2p["fixed"]

# H2 storage (tank type)
store = dict(capex=15_000, life=30, eta=0.98, loss=2e-5)
store["capital_cost"] = store["capex"] * annuity(WACC, store["life"])

In [ ]:
n.add("Store", "IDNSM_h2_store",
    bus="IDNSM_H2",
    carrier="H2",
    e_nom=0,
    e_nom_extendable=True,
    efficiency=store["eta"],
    standing_loss=store["loss"],
    capital_cost=store["capital_cost"],
    e_cyclic=True)


n.add("Load", name,
    bus=h2_bus,
    carrier="H2",
    p_set=0.0)


Index(['IDNSM_h2_store'], dtype='object')

In [24]:
# print(n.carriers)
# print(n.buses[["x", "y"]])
print(n.links)
print(n.links.loc["IDNSM_ely"])
print(n.links.loc["IDNJW-IDNNU-ext-2023"])
print(n.links.loc["IDNJW-IDNNU-ext-2035"])

                          bus0      bus1    type                     carrier  \
Link                                                                           
BRNXX-MYSSK-ext-2023     BRNXX     MYSSK    hvdc    interconnector-BRN-MYSSK   
MYSSK-BRNXX-ext-2023     MYSSK     BRNXX    hvdc    interconnector-BRN-MYSSK   
IDNJW-IDNKA-ext-2023     IDNJW     IDNKA  subsea  interconnector-IDNJW-IDNKA   
IDNKA-IDNJW-ext-2023     IDNKA     IDNJW  subsea  interconnector-IDNJW-IDNKA   
IDNJW-IDNNU-ext-2023     IDNJW     IDNNU    hvdc  interconnector-IDNJW-IDNNU   
...                        ...       ...     ...                         ...   
SGPXX-VNMSO-ext-2035     SGPXX     VNMSO    hvdc    interconnector-VNMSO-SGP   
KHMXX-SGPXX-ext-2035     KHMXX     SGPXX    hvdc      interconnector-KHM-SGP   
SGPXX-KHMXX-ext-2035     SGPXX     KHMXX    hvdc      interconnector-KHM-SGP   
IDNSM_ely                IDNSM  IDNSM_H2                                  H2   
IDNSM_h2p             IDNSM_H2     IDNSM